# Well Name Cleaner

In [1]:
import pandas as pd
import pymssql as psql
# import cx_Oracle as cx
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from scipy import interpolate, integrate
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [120]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Data Directory
import os
username=os.getlogin()
data_dir='C:/Users/'+username+'/Documents/GitHub/well-name-cleaner/'
# plk_data_dir='C:/Users/'+username+'/Documents/Data Eval/'

In [4]:
# Update Queries
ansr = False

In [5]:
# Function for SQL to Dataframe
def getSQLData(conType, conString, columnNames, sqlString, dataDir, dataFile
               , updateLocal=False, indexCol=None, autoIndex=True):
    
    if updateLocal==True:
        if conType == 'SQL Server': 
            con=psql.connect(conString[0],conString[1],conString[2],conString[3])
        elif conType == 'Oracle':
            con=cx.connect(conString)
        cur=con.cursor()
        cur.execute(sqlString)
        fetchedData=cur.fetchall()
        result=pd.DataFrame(fetchedData,columns=columnNames)
        if not autoIndex:
            result.set_index(indexCol,inplace=True)
        cur.close()
        con.close()
        result.to_pickle(dataDir+dataFile)
    if updateLocal==False:
        try:
            result=pd.read_pickle(dataDir+dataFile)
            result.columns=columnNames
        except IOError:
            print('file '+dataFile+' not found')
            if conType == 'SQL Server': 
                con=psql.connect(conString[0],conString[1],conString[2],conString[3])
            elif conType == 'Oracle':
                con=cx.connect(conString)
            cur=con.cursor()
            cur.execute(sqlString)
            fetchedData=cur.fetchall()
            result=pd.DataFrame(fetchedData,columns=columnNames)
            if not autoIndex:
                result.set_index(indexCol,inplace=True)
            cur.close()
            con.close()
            result.to_pickle(dataDir+dataFile)
    return result

In [6]:
data_file='WellGenData.pkl'

con_type='SQL Server'

con_string='CKCWBDA2','','',''

col_names=[
        'API_NO14','WELL_AUTO_NAME','WELL_COMMON_NAME','WELL_COMP_NAME','RESERVOIR'
      ,'COMP_TYPE','WELL_STATUS','STATUS_DATE','MOP','WELL_ID'
      ,'COMPLETION_ID','WELLBORE_ID','COST_CENTER','OPER_NO','ORG_SEQNO'
      ,'RMT_TEAM','UNIT','GEO_OFFSET_EAST','GEO_OFFSET_NORTH','GEO_LATITUDE'
      ,'GEO_LONGITUDE','WOA','TANK_SETTING','SOURCE'
    ]
    
sql_string="""
    SELECT * FROM [BDADB].[dbo].[tbl_Well_Info_General_Detail]
    """

dataWellInfo=getSQLData(con_type,con_string,col_names,sql_string,data_dir,data_file,updateLocal=ansr)
print('collected well general info')

file WellGenData.pkl not found
collected well general info


In [7]:
data_file='WellXSPOCTestData.pkl'

con_type='SQL Server'

con_string='CKCWBDA2','','',''

col_names=[
        'NodeID','TestDate','Duration','OilRate','WaterRate'
      ,'GasRate','TubingPressure','CasingPressure','Runtime','StrokeLength'
      ,'FluidAbovePump','Pumpsize','SPM','FlowlineTemp','CasingTemp'
      ,'WaterSG','OilAPIGravity','Approved','DateModified','UserID'
      ,'CO2','FacilityName','SeparatorNumber','ZoneCode','TestCode'
      ,'TotalFluidRate','WaterCut','HPGas','LPGas','CasingGas'
      ,'GasLiftGas','GOR','PumpSpeed','FlowLinePressure','TubingTemp'
      ,'PumpEfficiency','ChokeSize','WellTestFailCode','WellTestFailReason','PumpIntakePressure'
      ,'PumpDischargePressure','PumpIntakeTemp','MotorTemp','DriveCurrent','DriveSpeed'
      ,'RodRPM','RodTorque','TriplexDischargePressure','SuctionPressure','LinePressureToSetting'
      ,'TestComment','ExportFlag','HighPressureGasRate','CasingGasRate','GasLiftGasRate'
      ,'WaterCutMethod','WaterCutSource','TesterType','Source'
    ]
    
sql_string="""
    SELECT * FROM [BDADB].[dbo].[tbl_Well_XSPOC_Test_Hist]
    WHERE TestDate >= '01/01/2018'
    """

dataXSPOC=getSQLData(con_type,con_string,col_names,sql_string,data_dir,data_file,updateLocal=ansr)
print('collected XSPOC test info')

file WellXSPOCTestData.pkl not found
collected XSPOC test info


In [9]:
# dataPI=pd.read_csv(data_dir+'PI Well Testing.csv')
# print('collected process instruments info')

In [10]:
dataLab=pd.read_csv(data_dir+'Lab Well Testing.csv', dtype={'Total Cut':np.float64})
print('collected Lab Cut info')

collected Lab Cut info


C:\Users\robinsoe\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
dataPROS=pd.read_csv(data_dir+'PROS Well Testing.csv')
print('collected PROS info')

collected PROS info


## Evaluated Data

In [62]:
# dataWellInfo

In [77]:
# dataXSPOC

In [64]:
# dataLab

In [65]:
# dataPROS

In [104]:
WellInfo_LH = dataWellInfo[(dataWellInfo['RMT_TEAM']=='LOST HILLS AREA') & (~dataWellInfo['WELL_STATUS'].isnull())]

In [106]:
WellInfo_LH.head(100)

,API_NO14,WELL_AUTO_NAME,WELL_COMMON_NAME,WELL_COMP_NAME,RESERVOIR,COMP_TYPE,WELL_STATUS,STATUS_DATE,MOP,WELL_ID,COMPLETION_ID,WELLBORE_ID,COST_CENTER,OPER_NO,ORG_SEQNO,RMT_TEAM,UNIT,GEO_OFFSET_EAST,GEO_OFFSET_NORTH,GEO_LATITUDE,GEO_LONGITUDE,WOA,TANK_SETTING,SOURCE
306,04030137310100,11EEW-H21RD1,SECTION 11 EEW-H21,SECTION 11 EEW-H21 RD1,ETCH/W,PROD_OIL,INACTIVE,2014-05-07 00:00:00.0000000,PROG_CAVITY,GsOL7Yyj5a,QReiwlGlGi,5mIvNqtFmm,70400160.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.471014e+06,799337.635475,35.683474,-119.781124,None,None,EKPSPP
10540,04030537160000,CH11GE-A4I,SECTION 11 GE-A4i,SECTION 11 GE-A4I,ETCHGON,INJ_STEAM,ACTIVE,2017-07-13 00:00:00.0000000,NA,rJLKDNjkNc,pb95PSJYwv,MswRIP1PGY,70401308.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.467664e+06,800873.291743,35.687528,-119.792493,LambM,None,EKPSPP
10541,04030537200000,CH11GE-D8I,SECTION 11 GE-D8i,SECTION 11 GE-D8I,TULAREE,INJ_STEAM,INACTIVE,2016-04-20 00:00:00.0000000,NA,EemJELrDY9,SoT9X72Jq7,uje5K8raPq,70401308.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.468326e+06,800208.860415,35.685736,-119.790224,LambM,None,EKPSPP
10542,04030537340000,CH11GE-E9,SECTION 11 GE-E9,SECTION 11 GE-E9,ETCHGON,OIL_CYCL,INACTIVE,2015-11-18 00:00:00.0000000,BEAM_POC,f5cqyC02pc,Yt33zv6RdW,32mUaZ3xse,70401308.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.468488e+06,800044.080085,35.685291,-119.789670,LambM,GROUP LINE,EKPSPP
10543,04030342630000,CH12DT-F1,SECTION 12 DT-F1,SECTION 12 DT-F1,TULARE,OIL_CYCL,ACTIVE,2018-11-02 00:00:00.0000000,BEAM_POC,kgcrmnwITD,CEXrnQfdsh,XeYc91cHP9,70401305.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.472274e+06,799743.230486,35.684649,-119.776907,LambM,SEC 12 A1,EKPSPP
10544,04030588530000,CH12DT-G1A,SECTION 12 DT-G1A,SECTION 12 DT-G1A,TULARE,OIL_CYCL,ACTIVE,2015-08-21 00:00:00.0000000,BEAM_POC,caDmltQbW9,8HRvOzNue2,dBM4c7yM1M,70401305.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.472198e+06,799644.669286,35.684375,-119.777157,LambM,SEC 12 A1,EKPSPP
10545,04030529490000,CH2ET-H14A,SECTION 2 ET-H14A,SECTION 2 ET-H14A,TULARE,OIL_CYCL,ACTIVE,2016-05-25 00:00:00.0000000,BEAM_POC,PBonc4iaR5,C2GJDyeM0Y,gDrugN7njR,70401300.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.469671e+06,804750.839498,35.698276,-119.785970,DillinB,SEC 2 J2,EKPSPP
10546,04030483390000,CH2ET-H14B,SECTION 2 ET-H14B,SECTION 2 ET-H14B,TULARE,OIL_CYCL,ACTIVE,2013-07-22 00:00:00.0000000,BEAM_POC,DbNXIyfwIe,JaK3kxkdSE,dVVL1hqLFH,70401300.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.469481e+06,804645.859289,35.697979,-119.786604,DillinB,SEC 2 J2,EKPSPP
10547,04030577570000,CH2ET-I10A,SECTION 2 ET-I10A,SECTION 2 ET-I10A,TULARE,OIL_CYCL,ACTIVE,2015-06-04 00:00:00.0000000,BEAM_POC,NP4cbYQUoO,OjXcy4KcCD,650IxgKyYt,70401300.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.468772e+06,804520.319037,35.697599,-119.788983,DillinB,SEC 2 E1,EKPSPP
10548,04030369690000,CH2ET-M17,SECTION 2 ET-M17,SECTION 2 ET-M17,TULARE,OIL_CYCL,INACTIVE,2016-08-02 00:00:00.0000000,BEAM_POC,tYIbjwzmNK,1JAomq4OTW,oNAs6fYDFP,70401300.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.470231e+06,803643.770488,35.695263,-119.784017,None,None,EKPSPP


In [107]:
PROS_LH = dataPROS[(dataPROS['Lease']=='Lost Hills')]

In [146]:
PROS_LH.head(10)

,Company,WellName,Lease,ObjectType,Separator,TestDate,BSW,Oil Gravity,Water Gravity,% Sand - Start,Separator PSI,Separator Temp,BPD-G,BPD-W,BPD-N,Well Status,Strokes Per Minute,Stroke Length,Choke 64ths - Start,TBG Temp - Start,TBG PSIG,CSG PSIG - Start,Line PSIG - Start,Flow - MCFD,GOR,TestType,RequestersName,Request Date
0,California Heavy Oil,2 JT-F11,Lost Hills,2 Phase 24 Hour,20528-228-41S,9/20/2013 0:00,56.68,14.7,9.8,0.0,137.370,83,1.61,0.91,0.7,Rod Pump,8.18,37,NaN,96,95,-4 PSI/90*F,90,0.0,NaN,Gross,NaN,NaN
1,California Heavy Oil,2 BEW-O5,Lost Hills,2 Phase 24 Hour,20330-228-41S,9/20/2013 0:00,96.43,NaN,7.7,0.0,153.370,96,674.56,650.48,24.08,Rod Pump,10.33,100,NaN,99,110,-4 PSI/96*F,60,0.0,NaN,Gross,NaN,NaN
2,California Heavy Oil,2 ET-J12,Lost Hills,2 Phase 24 Hour,20274-270-40S,9/20/2013 0:00,86.67,NaN,8.9,0.0,125.720,97,21.83,18.92,2.91,Rod Pump,7.8,48,NaN,169,95,2 PSI/206*F,60,0.0,NaN,Gross,NaN,NaN
4,California Heavy Oil,12 AT-L5,Lost Hills,2 Phase 24 Hour,20288-228-40S,9/20/2013 0:00,62.36,11.9,8.6,0.0,131.070,88,18.74,11.69,7.05,Rod Pump,7.87,54,NaN,147,295,-1.5 PSI / 103*F,76,0.0,NaN,Gross,NaN,NaN
5,California Heavy Oil,12 AT-J4,Lost Hills,2 Phase 24 Hour,20288-228-40S,9/20/2013 0:00,59.95,11.7,8.5,0.0,142.070,98,4.97,2.98,1.99,Rod Pump,2.28,54,NaN,145,160,1 PSI / 204*F,76,0.0,NaN,Gross,NaN,NaN
6,California Heavy Oil,2 AT-R11,Lost Hills,2 Phase 24 Hour,20247-228-41S,9/20/2013 0:00,60.39,11.2,8.5,0.0,114.900,101,26.26,15.86,10.4,Rod Pump,7.06,54,NaN,146,120,-1.5 PSI / 203*F,50,0.0,NaN,Gross,NaN,NaN
7,California Heavy Oil,2 AT-R13A,Lost Hills,2 Phase 24 Hour,20247-228-41S,9/21/2013 0:00,53.08,14.3,9.4,0.0,111.300,78,45.45,24.12,21.33,Rod Pump,6.32,68,NaN,178,100,4 PSI / 72*F,50,0.0,NaN,Gross,NaN,NaN
8,California Heavy Oil,11 AT-F24,Lost Hills,2 Phase 24 Hour,20298-228-41S,9/20/2013 0:00,66.36,11.4,8.6,0.0,122.170,122,67.34,44.69,22.65,Rod Pump,7.21,"48""",NaN,177,105,2 PSI / 205*F,72,0.0,NaN,Gross,NaN,NaN
9,California Heavy Oil,2 AT-N9A,Lost Hills,2 Phase 24 Hour,20298-228-41S,9/21/2013 0:00,62.52,10.9,8.5,0.0,122.170,114,32.21,20.14,12.07,Rod Pump,6.12,66,NaN,189,100,3.5 PSI / 219*F,50,0.0,NaN,Gross,NaN,NaN
10,California Heavy Oil,2 LT-B2B,Lost Hills,2 Phase 24 Hour,50528-228-41S,9/20/2013 0:00,68.75,2.6,9.7,NaN,90.895,107,45.00,30.94,14.06,Rod Pump,7.88,48,NaN,192,110,-4 PSI/190*F,80,0.0,NaN,Gross,NaN,NaN


In [125]:
welllist = pd.Series(PROS_LH['WellName'].unique()).sort_values()

In [135]:
welllist[welllist == '11 FE-I17']

720    11 FE-I17
dtype: object

In [127]:
welllist2 = pd.Series(WellInfo_LH['WELL_COMP_NAME'].unique()).sort_values()

In [137]:
welllist2[welllist2 == 'SECTION 11 FE-I17']

800    SECTION 11 FE-I17
dtype: object

In [128]:
welllist2.sort_values()

1095                      E-D24
1096                      E-G15
980                      E-I22A
786                       E-M20
863      SECTION 11 AE-E21 (EW)
959          SECTION 11 AEW-A21
837          SECTION 11 AEW-C21
729          SECTION 11 AEW-D21
880          SECTION 11 AEW-F21
1141         SECTION 11 AEW-F24
761           SECTION 11 AT-A20
864           SECTION 11 AT-A23
915          SECTION 11 AT-A23B
838          SECTION 11 AT-A23C
881           SECTION 11 AT-A24
960          SECTION 11 AT-A24A
730          SECTION 11 AT-A24B
674           SECTION 11 AT-A25
961           SECTION 11 AT-A26
916           SECTION 11 AT-B20
731          SECTION 11 AT-B20A
692           SECTION 11 AT-B22
962           SECTION 11 AT-B23
839           SECTION 11 AT-B25
762          SECTION 11 AT-B25A
732           SECTION 11 AT-B26
895           SECTION 11 AT-C20
1142          SECTION 11 AT-C21
1143          SECTION 11 AT-C23
763          SECTION 11 AT-C23A
693           SECTION 11 AT-C24
917     

In [109]:
XSPOC_LH = pd.merge(dataXSPOC, WellInfo_LH, how='left', left_on='NodeID', right_on='WELL_AUTO_NAME')

In [110]:
XSPOC_LH.head(1)

,NodeID,TestDate,Duration,OilRate,WaterRate,GasRate,TubingPressure,CasingPressure,Runtime,StrokeLength,FluidAbovePump,Pumpsize,SPM,FlowlineTemp,CasingTemp,WaterSG,OilAPIGravity,Approved,DateModified,UserID,CO2,FacilityName,SeparatorNumber,ZoneCode,TestCode,TotalFluidRate,WaterCut,HPGas,LPGas,CasingGas,GasLiftGas,GOR,PumpSpeed,FlowLinePressure,TubingTemp,PumpEfficiency,ChokeSize,WellTestFailCode,WellTestFailReason,PumpIntakePressure,PumpDischargePressure,PumpIntakeTemp,MotorTemp,DriveCurrent,DriveSpeed,RodRPM,RodTorque,TriplexDischargePressure,SuctionPressure,LinePressureToSetting,TestComment,ExportFlag,HighPressureGasRate,CasingGasRate,GasLiftGasRate,WaterCutMethod,WaterCutSource,TesterType,Source,API_NO14,WELL_AUTO_NAME,WELL_COMMON_NAME,WELL_COMP_NAME,RESERVOIR,COMP_TYPE,WELL_STATUS,STATUS_DATE,MOP,WELL_ID,COMPLETION_ID,WELLBORE_ID,COST_CENTER,OPER_NO,ORG_SEQNO,RMT_TEAM,UNIT,GEO_OFFSET_EAST,GEO_OFFSET_NORTH,GEO_LATITUDE,GEO_LONGITUDE,WOA,TANK_SETTING,SOURCE
0,21-1B,2018-06-29 13:22:04,24.0,2.31,1.89,0.0,100.0,28.0,2.64,52.299999,NaN,1.5,5.8,0.0,0.0,1.02,24.0,False,2018-07-05 13:23:37.273,ca1-hergenrt,NaN,1-1BSOZ,0.0,57671.0,0.0,4.2,44.900002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,NaN,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,EKXSPOC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
XSPOC_LH.shape

(87742, 83)

In [112]:
merged_data = pd.merge(PROS_LH, WellInfo_LH, how='left', left_on='WellName', right_on='WELL_AUTO_NAME')

In [113]:
merged_data.head()

,Company,WellName,Lease,ObjectType,Separator,TestDate,BSW,Oil Gravity,Water Gravity,% Sand - Start,Separator PSI,Separator Temp,BPD-G,BPD-W,BPD-N,Well Status,Strokes Per Minute,Stroke Length,Choke 64ths - Start,TBG Temp - Start,TBG PSIG,CSG PSIG - Start,Line PSIG - Start,Flow - MCFD,GOR,TestType,RequestersName,Request Date,API_NO14,WELL_AUTO_NAME,WELL_COMMON_NAME,WELL_COMP_NAME,RESERVOIR,COMP_TYPE,WELL_STATUS,STATUS_DATE,MOP,WELL_ID,COMPLETION_ID,WELLBORE_ID,COST_CENTER,OPER_NO,ORG_SEQNO,RMT_TEAM,UNIT,GEO_OFFSET_EAST,GEO_OFFSET_NORTH,GEO_LATITUDE,GEO_LONGITUDE,WOA,TANK_SETTING,SOURCE
0,California Heavy Oil,2 JT-F11,Lost Hills,2 Phase 24 Hour,20528-228-41S,9/20/2013 0:00,56.68,14.7,9.8,0.0,137.37,83,1.61,0.91,0.7,Rod Pump,8.18,37,NaN,96,95,-4 PSI/90*F,90,0.0,NaN,Gross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,California Heavy Oil,2 BEW-O5,Lost Hills,2 Phase 24 Hour,20330-228-41S,9/20/2013 0:00,96.43,NaN,7.7,0.0,153.37,96,674.56,650.48,24.08,Rod Pump,10.33,100,NaN,99,110,-4 PSI/96*F,60,0.0,NaN,Gross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,California Heavy Oil,2 ET-J12,Lost Hills,2 Phase 24 Hour,20274-270-40S,9/20/2013 0:00,86.67,NaN,8.9,0.0,125.72,97,21.83,18.92,2.91,Rod Pump,7.8,48,NaN,169,95,2 PSI/206*F,60,0.0,NaN,Gross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,California Heavy Oil,12 AT-L5,Lost Hills,2 Phase 24 Hour,20288-228-40S,9/20/2013 0:00,62.36,11.9,8.6,0.0,131.07,88,18.74,11.69,7.05,Rod Pump,7.87,54,NaN,147,295,-1.5 PSI / 103*F,76,0.0,NaN,Gross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California Heavy Oil,12 AT-J4,Lost Hills,2 Phase 24 Hour,20288-228-40S,9/20/2013 0:00,59.95,11.7,8.5,0.0,142.07,98,4.97,2.98,1.99,Rod Pump,2.28,54,NaN,145,160,1 PSI / 204*F,76,0.0,NaN,Gross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
merged_data['API_NO14'].nunique()

0

In [26]:
dataPROS['WellName'].nunique()

1513

In [33]:
# Carry = data['TestComment'][data['TestComment'].str.contains('carried|Carried|CARRIED|CC |cc |inf |Inf |INF |IP | IP ')]
# Carry = pd.DataFrame(Carry)
# Carry.count()

In [34]:
# pd.set_option('display.max_colwidth', -1)
# Carry[Carry['TestComment'].str.contains('PIP')]

In [144]:
def add_API(row, WellInfo_LH):
    
    auto_list = WellInfo_LH[WellInfo_LH['WELL_AUTO_NAME']==row['WellName']]
    common_list = WellInfo_LH[WellInfo_LH['WELL_COMMON_NAME']==row['WellName']]
    comp_list = WellInfo_LH[WellInfo_LH['WELL_COMP_NAME']==row['WellName']]
    
    if row['WellName'] in auto_list:
        API = WellInfo_LH['API_NO14']
        
    elif row['WellName'] in common_list:
        API = WellInfo_LH['API_NO14']
        
    elif row['WellName'] == comp_list:
        API = WellInfo_LH['API_NO14']
        
    else:
        API = None
    
    return API

In [151]:
WellInfo_LH[WellInfo_LH['WELL_COMP_NAME']=='SECTION 11 AEW-F24']

,API_NO14,WELL_AUTO_NAME,WELL_COMMON_NAME,WELL_COMP_NAME,RESERVOIR,COMP_TYPE,WELL_STATUS,STATUS_DATE,MOP,WELL_ID,COMPLETION_ID,WELLBORE_ID,COST_CENTER,OPER_NO,ORG_SEQNO,RMT_TEAM,UNIT,GEO_OFFSET_EAST,GEO_OFFSET_NORTH,GEO_LATITUDE,GEO_LONGITUDE,WOA,TANK_SETTING,SOURCE
27592,04030448560000,THE11AEW-F24,SECTION 11 AEW-F24,SECTION 11 AEW-F24,ETCH/W,OIL_CYCL,ACTIVE,2011-11-16 00:00:00.0000000,BEAM_POC,7deR7rBGf2,8yfHQ7pXjv,T4MHxywOWA,70401315.0,999115.0,1001010301,LOST HILLS AREA,NON UNIT,1.471579e+06,799833.910865,35.684864,-119.77925,LambM,SEC 11 A1,EKPSPP


In [153]:
PROS_LH[PROS_LH['WellName']=='11 AEW-F24']

,Company,WellName,Lease,ObjectType,Separator,TestDate,BSW,Oil Gravity,Water Gravity,% Sand - Start,Separator PSI,Separator Temp,BPD-G,BPD-W,BPD-N,Well Status,Strokes Per Minute,Stroke Length,Choke 64ths - Start,TBG Temp - Start,TBG PSIG,CSG PSIG - Start,Line PSIG - Start,Flow - MCFD,GOR,TestType,RequestersName,Request Date
210,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20247-228-41S,8/23/2013 0:00,91.92,NaN,9.1,0.0,135.42625,114,175.57,161.38,14.19,Rod Pump,6.8,120,NaN,142,110,8 PSI / 116*F,72,0.0,NaN,Gross,NaN,NaN
909,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20247-228-41S,12/2/2013 0:00,93.75,NaN,9.4,0.0,143.27625,86,173.67,162.82,10.85,Rod Pump,9.23,120,NaN,121,115,-.5 PSI / 109*F,76,0.0,NaN,Gross,NaN,NaN
1028,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20218-228-42S,1/23/2014 0:00,96.32,NaN,8.8,0.0,131.57625,98,311.00,299.56,11.44,Rod Pump,9.22,120,NaN,113,110,28 PSI / 46*F,80,0.0,NaN,Gross,NaN,NaN
1172,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20288-228-40S,9/25/2013 0:00,92.71,NaN,9.0,0.0,144.88250,98,195.51,181.26,14.25,Rod Pump,6.78,120,NaN,128,110,7 PSI / 101*F,72,0.0,NaN,Gross,NaN,NaN
1201,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20298-228-41S,9/17/2013 0:00,94.55,NaN,9.0,0.0,131.57625,106,169.88,160.62,9.26,Rod Pump,6.78,120,NaN,125,110,10 PSI / 102*F,72,0.0,NaN,Gross,NaN,NaN
1257,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20202-250-34S,11/12/2013 0:00,96.69,NaN,9.2,0.0,225.37000,112,207.08,200.23,6.85,Rod Pump,9.24,120,NaN,116,110,1 PSI / 91*F,76,0.0,NaN,Gross,NaN,NaN
1560,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20207-270-40S,3/3/2017 0:00,94.44,NaN,9.3,0.0,143.27625,61,65.91,62.25,3.66,Rod Pump,9.36,120,NaN,106,120,2.5 PSI / 63*F,90,0.0,NaN,Gross,NaN,NaN
1847,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20247-228-41S,3/15/2013 0:00,90.22,NaN,7.8,0.0,114.87000,70,47.95,43.26,4.69,Rod Pump,6.81,121,NaN,130,80,-3 PSI/110*F,64,0.0,NaN,Gross,NaN,NaN
2560,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20218-228-42S,5/28/2013 0:00,96.15,NaN,8.5,0.0,116.67625,152,302.50,290.85,11.65,Rod Pump,6.78,120,NaN,218,130,1 PSI / 213*F,65,0.0,NaN,Gross,NaN,NaN
2621,California Heavy Oil,11 AEW-F24,Lost Hills,2 Phase 24 Hour,20218-228-42S,6/1/2013 0:00,97.37,NaN,8.6,0.0,122.17000,182,235.33,229.14,6.19,Rod Pump,6.79,120,NaN,212,90,2.5 PSI / 218*F,65,0.0,NaN,Gross,NaN,NaN


In [145]:
PROS_LH['API'] = PROS_LH.apply(lambda row: add_API(row, WellInfo_LH), axis=1)

KeyError: ('WellName', 'occurred at index Company')